# Mini Pytato 

In [8]:
import numpy as np
import numpy.linalg as la
import pymbolic.primitives as p
import loopy as lp
import pyopencl as cl
import pyopencl.array

%load_ext gvmagic

ctx = cl.create_some_context(interactive=True)
queue = cl.CommandQueue(ctx)

In this demo, we will deal with numpy-ish arrays that *all* have shape `(10, 10)` (think of them as $10\times 10$ matrices maybe) and contain floating point numbers.

## Building Expression Graphs

We would like to build a data structure that represents the following computation, so that:

- we can execute it later
- we can generate code for it

Reflect for a moment what `result` *is*. Does it contain data? What is its "meaning"?

(`Placeholder` is not yet implemented. We will do this in the next cell. This cell is repeated below for convenience.)

In [9]:
a = Placeholder("a")
b = Placeholder("b")

result = a + b * a

Now let's implement the `Array` base class, along with `Sum`, `Product`, and `Placeholder` subclasses. To do so, fill in code for the `...` ellipses:

In [15]:
class Array:
    def __init__(self):
        self.shape = (10, 10)
        self.dtype = np.float64
        
    def __add__(self, other):
        ...
    
    def __mul__(self, other):
        ...
        
class Sum(Array):
    def __init__(self, a, b):
        super().__init__()
        ...
        
class Product(Array):
    def __init__(self, a, b):
        super().__init__()
        ...
        
class Placeholder(Array):
    def __init__(self, name):
        super().__init__()
        ...


In [15]:
# expand this cell for solution
class Array:
    def __init__(self):
        self.shape = (10, 10)
        self.dtype = np.float64
        
    def __add__(self, other):
        return Sum(self, other)
    
    def __mul__(self, other):
        return Product(self, other)
        
class Sum(Array):
    def __init__(self, a, b):
        super().__init__()
        self.a = a
        self.b = b
        
class Product(Array):
    def __init__(self, a, b):
        super().__init__()
        self.a = a
        self.b = b
        
class Placeholder(Array):
    def __init__(self, name):
        super().__init__()
        self.name = name


Here is the cell from above once again:

In [16]:
a = Placeholder("a")
b = Placeholder("b")

result = a + b * a

Can you visualize the data structure that we have just created? (Execute this cell to see if your mental image was correct.)

In [17]:
%%dot
digraph {
    sum -> prod;
    sum -> a;
    prod -> a;
    prod -> b;
}

## Evaluating Expressions

Write code to evaluate the expression, provided some values for `a` and `b`:

In [19]:
def evaluate_expr(expr, values):
    if isinstance(expr, Placeholder):
        ...
    elif isinstance(expr, Sum):
        ...
    ...
    else:
        raise ValueError(f"unexpected node type: {expr.__class__}")
    

In [19]:
# expand this cell for solution
def evaluate_expr(expr, values):
    if isinstance(expr, Placeholder):
        return values[expr.name]
    elif isinstance(expr, Sum):
        return evaluate_expr(expr.a, values) + evaluate_expr(expr.b, values)
    elif isinstance(expr, Product):
        return evaluate_expr(expr.a, values) * evaluate_expr(expr.b, values)
    else:
        raise ValueError(f"unexpected node type: {expr.__class__}")
    

Let's test if we got it right. If all is well, this should produce an array of zeroes:

In [20]:
a_val = np.random.randn(10, 10)
b_val = np.random.randn(10, 10)

evaluate_expr(result, {"a": a_val, "b": b_val}) - (a_val + b_val * a_val)

Functions like `evaluate_expr` are hard to extend once they're written. Using a class with the "visitor pattern" can help, where we make one method per node type that needs to be handled. Make note of the `rec` method that dispatches each node to the appropriate method.

Fill in the missing method implementations:

In [37]:
class EvaluationMapper:
    def __init__(self, values):
        self.values = values

    def rec(self, expr):
        method = getattr(self, f"map_{expr.__class__.__name__}")
        return method(expr)

    def map_Sum(self, expr):
        ...
    
    def map_Product(self, expr):
        ...

    def map_Placeholder(self, expr):
        ...


In [37]:
# expand this cell for solution
class EvaluationMapper:
    def __init__(self, values):
        self.values = values

    def rec(self, expr):
        method = getattr(self, f"map_{expr.__class__.__name__}")
        return method(expr)

    def map_Sum(self, expr):
        return self.rec(expr.a) + self.rec(expr.b)
    
    def map_Product(self, expr):
        return self.rec(expr.a) * self.rec(expr.b)

    def map_Placeholder(self, expr):
        return self.values[expr.name]


Again, let's test that this does what we intend:

In [38]:
EvaluationMapper({"a": a_val, "b": b_val}).rec(result) - (a_val + b_val * a_val)

## Generating code

To generate code using our code generator Loopy, all we need to do is to transcribe our array-valued expression into a scalar one, using an existing expression tree library called `pymbolic`. We have imported this as `p` above. The equivalents of `Placeholder`s there are called `Variable`. Let's write a mapper that does this. Again, fill in the blanks:

In [63]:
class CodegenMapper:
    def rec(self, expr):
        method = getattr(self, f"map_{expr.__class__.__name__}")
        return method(expr)
        
    def map_Placeholder(self, expr):
        return p.Variable(expr.name)[p.Variable("i"), p.Variable("j")]

    def map_Sum(self, expr):
        ...
    
    def map_Product(self, expr):
        ...


In [63]:
# expand this cell for solution
class CodegenMapper:
    def rec(self, expr):
        method = getattr(self, f"map_{expr.__class__.__name__}")
        return method(expr)
        
    def map_Placeholder(self, expr):
        return p.Variable(expr.name)[p.Variable("i"), p.Variable("j")]

    def map_Sum(self, expr):
        return self.rec(expr.a) + self.rec(expr.b)
    
    def map_Product(self, expr):
        return self.rec(expr.a) * self.rec(expr.b)


Let's with some expressions:

In [64]:
x = Placeholder("x")
y = Placeholder("y")

expr = (x+x*y)*x

# expr = (x+y)
# expr = expr*expr
# expr = expr*expr
# expr = expr*expr
# expr = expr*expr
# expr = expr*expr

Generate code for these expressions:

In [65]:
# expand this cell for solution
print(CodegenMapper().rec(expr))

Let's make a loopy kernel for the expression. Fill in the generated scalar expression for the RHS of the `Assignment`:

In [66]:
knl = lp.make_kernel(
    "{[i,j]: 0<=i,j<10}",
    [lp.Assignment(
        p.Variable("lhs")[p.Variable("i"), p.Variable("j")], 
        ...
    )])
print(knl)

In [66]:
# expand this cell for solution
knl = lp.make_kernel(
    "{[i,j]: 0<=i,j<10}",
    [lp.Assignment(
        p.Variable("lhs")[p.Variable("i"), p.Variable("j")], 
        CodegenMapper().rec(expr)
    )])
print(knl)

Next, let's run the code on our OpenCL device:

In [67]:
xval = np.random.randn(10, 10)
yval = np.random.randn(10, 10)

evt, (res,) = knl(queue, x=xval, y=yval)

Check the result:

In [68]:
print(la.norm(res- (xval+xval*yval)*xval))

Look at the generated C code:

In [69]:
knl = lp.add_and_infer_dtypes(knl, {"x": xval.dtype, "y": yval.dtype})

code = lp.generate_code_v2(knl).device_code()
print(code)

And this is where you might start transforming the loopy code. Here is a simple example that tiles the loop:

In [76]:
tiled = lp.split_iname(knl, "i", 4)
tiled = lp.split_iname(tiled, "j", 4)
tiled = lp.prioritize_loops(tiled, "i_outer, j_outer, i_inner, j_inner")

code = lp.generate_code_v2(tiled).device_code()
print(code)